In [ ]:
import numpy as np
import pandas as pd
import requests
import json

In [ ]:
url = "https://data.iowa.gov/resource/m3tr-qhgy.csv"
alc_df = pd.read_csv(url)


In [ ]:
def return_name(string):
  return " ".join(string.split(" ")[:-1])

pop_df= pd.read_csv("https://data.iowa.gov/resource/qtnr-zsrc.csv")
pop_df["county"] = pop_df["geographicname"].apply(return_name).str.upper()


In [ ]:
def return_year(string):
  return string.split("-")[0]

pop_df["year"] = pop_df["year"].apply(return_year)


In [ ]:
alc_df["Year"] = alc_df["date"].apply(return_year)
alc_df["county"] = alc_df["county"].str.upper()


In [ ]:

alc_df["county"].unique()

array(['JOHNSON', 'POLK', 'IOWA', 'CERRO GORDO', 'STORY', 'BOONE',
       'SCOTT', 'DICKINSON', 'LINN', 'DUBUQUE', 'FLOYD', 'MAHASKA',
       'MARSHALL', 'WORTH', 'BLACK HAWK', 'MADISON', 'WINNESHIEK',
       'BENTON', 'DALLAS', 'SAC', 'CLAYTON', 'APPANOOSE', "O'BRIEN",
       'CHICKASAW', 'WOODBURY', 'POTTAWATTAMIE', 'BREMER', 'LUCAS',
       'BUCHANAN', 'POCAHONTAS', 'DES MOINES', 'POWESHIEK', 'WAPELLO',
       'FAYETTE', 'MILLS', 'PLYMOUTH', 'WARREN', 'CARROLL', 'DECATUR',
       'WASHINGTON', 'SHELBY', 'MARION', 'CEDAR', 'HARDIN', 'ALLAMAKEE',
       'JACKSON', 'MUSCATINE', 'HENRY', 'KOSSUTH', 'DAVIS', 'JASPER',
       'GREENE', 'VAN BUREN', 'TAMA', 'HARRISON', 'CHEROKEE', 'JONES',
       'MONONA', 'BUTLER', 'ADAIR', 'CLAY', 'WEBSTER', nan, 'EMMET',
       'WINNEBAGO', 'DELAWARE', 'CLINTON', 'MONTGOMERY', 'UNION', 'LEE',
       'SIOUX', 'BUENA VISTA', 'PALO ALTO', 'MITCHELL', 'CLARKE',
       'WRIGHT', 'CASS', 'PAGE', 'HAMILTON', 'GRUNDY', 'HUMBOLDT',
       'JEFFERSON', 'IDA', 'HO

In [ ]:
pop_df["county"].unique()

array(['STORY', 'FAYETTE', 'HOWARD', 'APPANOOSE', 'CLAYTON', 'JOHNSON',
       'MONONA', 'SIOUX', 'JEFFERSON', 'LINN', 'RINGGOLD', 'POCAHONTAS',
       'POLK', 'HANCOCK', 'WEBSTER', 'TAMA', 'CEDAR', "O'BRIEN", 'EMMET',
       'GUTHRIE', 'PALO ALTO', 'CHICKASAW', 'OSCEOLA', 'MONTGOMERY',
       'BLACK HAWK', 'WASHINGTON', 'LYON', 'LUCAS', 'DICKINSON', 'MILLS',
       'BENTON', 'DUBUQUE', 'JONES', 'PAGE', 'SCOTT', 'WAYNE',
       'VAN BUREN', 'CRAWFORD', 'POTTAWATTAMIE', 'LEE', 'CHEROKEE',
       'TAYLOR', 'GREENE', 'MAHASKA', 'HARRISON', 'HUMBOLDT',
       'CERRO GORDO', 'CALHOUN', 'UNION', 'WINNEBAGO', 'MITCHELL',
       'DAVIS', 'DALLAS', 'KOSSUTH', 'JACKSON', 'ALLAMAKEE', 'BUTLER',
       'MUSCATINE', 'IOWA', 'ADAIR', 'DELAWARE', 'FLOYD', 'WARREN',
       'WORTH', 'AUDUBON', 'MADISON', 'CLAY', 'BREMER', 'DECATUR',
       'LOUISA', 'GRUNDY', 'WOODBURY', 'FRANKLIN', 'BUENA VISTA',
       'CARROLL', 'KEOKUK', 'CLARKE', 'SHELBY', 'SAC', 'JASPER',
       'POWESHIEK', 'HENRY', 'CLINTON', '

In [ ]:
df = pd.read_html("https://www.iowa-demographics.com/counties_by_population")[0]
df["year"] = "2021"
df["county"] = df["County"].apply(return_name).str.upper()
df["population"] = df["Population"]
df = df[["county", "population", "year"]]
df

,county,population,year
0,POLK,496844,2021
1,LINN,228939,2021
2,SCOTT,174170,2021
3,JOHNSON,154748,2021
4,BLACK HAWK,130368,2021
...,...,...,...
95,TAYLOR,5868,2021
96,AUDUBON,5635,2021
97,RINGGOLD,4639,2021
98,ADAMS,3641,2021


In [ ]:
pop_df = pop_df[["county", "population", "year"]]
pop_df = pd.concat([pop_df, df])

In [ ]:
combined = alc_df.merge(pop_df, how = "left")
combined[combined["year"].equals(combined["Year"])]

KeyError: ignored